In [1]:
import pandas as pd
import numpy as np
import keras
import h5py
from keras.models import load_model
import pandas as pd
from joblib import dump
from joblib import load
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.backend import manual_variable_initialization
from keras.models import load_model


Using TensorFlow backend.


In [2]:
from numpy.random import seed
seed(1) #seed fixing for keras
from tensorflow import set_random_seed
set_random_seed(2)

In [3]:
all_data_df = pd.read_csv("../Data/all_data.csv")

In [4]:
all_data = pd.get_dummies(all_data_df, columns = ['Race', 'Sex', 'State'])

In [5]:
all_data_clean = all_data.drop(columns=['Unnamed: 0', 'County Code', 'Year', 'Year Code', 'States Code', 'Sex Code', 'Race Code', 'Cancer Sites Code', 'Count', 'Population'])

In [6]:
all_data_dropped = all_data_clean.dropna(subset=["Cancer Sites"])

In [7]:
X = all_data_dropped.drop('Cancer Sites', axis=1)
X = X.drop('Crude Rate', axis=1)
y = all_data_dropped['Cancer Sites']
print(X.shape, y.shape)

(6818, 66) (6818,)


In [8]:
X = X.fillna(0)
X = X.replace("Not Applicable", 0)
X = X.drop('State_PR', axis =1)

In [9]:
X.columns

Index(['CO', 'Lead', 'NO2', 'Ozone', 'PM10', 'PM2.5', 'SO2',
       'Race_American Indian or Alaska Native',
       'Race_Asian or Pacific Islander', 'Race_Black or African American',
       'Race_Other Races and Unknown combined', 'Race_White', 'Sex_Female',
       'Sex_Male', 'State_AK', 'State_AL', 'State_AR', 'State_AZ', 'State_CA',
       'State_CO', 'State_CT', 'State_DE', 'State_FL', 'State_GA', 'State_HI',
       'State_IA', 'State_ID', 'State_IL', 'State_IN', 'State_KS', 'State_KY',
       'State_LA', 'State_MA', 'State_MD', 'State_ME', 'State_MI', 'State_MN',
       'State_MO', 'State_MS', 'State_MT', 'State_NC', 'State_ND', 'State_NE',
       'State_NH', 'State_NJ', 'State_NM', 'State_NV', 'State_NY', 'State_OH',
       'State_OK', 'State_OR', 'State_PA', 'State_RI', 'State_SC', 'State_SD',
       'State_TN', 'State_TX', 'State_UT', 'State_VA', 'State_VI', 'State_VT',
       'State_WA', 'State_WI', 'State_WV', 'State_WY'],
      dtype='object')

In [10]:

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)




In [11]:
preprocess_pipeline = Pipeline(steps= [("transform", X_scaler)])

In [12]:
dump(preprocess_pipeline, '../Model_Info/pipeline.joblib')

['../Model_Info/pipeline.joblib']

In [13]:
from keras.models import Sequential
from keras.layers import Dense

# Create model and add layers
model = Sequential()
model.add(Dense(units=195, activation='relu', input_dim=65))
model.add(Dense(units=195, activation='relu'))
model.add(Dense(units=4, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [14]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=20,
    shuffle=True,
    verbose=2
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
 - 0s - loss: 0.8256 - acc: 0.5772
Epoch 2/20
 - 0s - loss: 0.7446 - acc: 0.5948
Epoch 3/20
 - 0s - loss: 0.7275 - acc: 0.5879
Epoch 4/20
 - 0s - loss: 0.7155 - acc: 0.5869
Epoch 5/20
 - 0s - loss: 0.7099 - acc: 0.5856
Epoch 6/20
 - 0s - loss: 0.7021 - acc: 0.5846
Epoch 7/20
 - 0s - loss: 0.7003 - acc: 0.5912
Epoch 8/20
 - 0s - loss: 0.6936 - acc: 0.5975
Epoch 9/20
 - 0s - loss: 0.6930 - acc: 0.5926
Epoch 10/20
 - 0s - loss: 0.6894 - acc: 0.5910
Epoch 11/20
 - 0s - loss: 0.6881 - acc: 0.5881
Epoch 12/20
 - 0s - loss: 0.6859 - acc: 0.5996
Epoch 13/20
 - 0s - loss: 0.6839 - acc: 0.6020
Epoch 14/20
 - 0s - loss: 0.6830 - acc: 0.6030
Epoch 15/20
 - 0s - loss: 0.6810 - acc: 0.6026
Epoch 16/20
 - 0s - loss: 0.6794 - acc: 0.6081
Epoch 17/20
 - 0s - loss: 0.6779 - acc: 0.6079
Epoch 18/20
 - 0s - loss: 0.6773 - acc: 0.6092
Epoch 19/20
 - 0s - loss: 0.6761 - acc: 0.6004
Epoch 20/20
 - 0s - loss: 0.6772 - acc: 0.6012


In [15]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 0.7776988447586463, Accuracy: 0.48328445747800586


In [16]:
encoded_predictions = model.predict_classes(X_test_scaled[:10])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [17]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:10])}")

Predicted classes: ['Lung and Bronchus' 'Lung and Bronchus' 'Lung and Bronchus'
 'Lung and Bronchus' 'Lung and Bronchus' 'Lung and Bronchus'
 'Lung and Bronchus' 'Lung and Bronchus' 'Larynx' 'Lung and Bronchus']
Actual Labels: ['Lung and Bronchus', 'Lung and Bronchus', 'Lung and Bronchus', 'Lung and Bronchus', 'Larynx', 'Lung and Bronchus', 'Larynx', 'Lung and Bronchus', 'Lung and Bronchus', 'Lung and Bronchus']


In [18]:
for i in range(len(X_test_scaled)):
    print("Prediction: " + str(model.predict([X_test_scaled[i].reshape(1,X_test_scaled[i].shape[0])])))


Prediction: [[1.0834939e-05 9.9998653e-01 2.5775441e-06 1.3205912e-12]]
Prediction: [[3.2221461e-03 9.9660254e-01 1.7534324e-04 8.1880600e-09]]
Prediction: [[6.1250306e-05 9.9993682e-01 1.8480526e-06 5.9297066e-13]]
Prediction: [[5.0580586e-12 1.0000000e+00 9.8957701e-21 2.2144877e-30]]
Prediction: [[0.25533321 0.27436486 0.2727485  0.19755341]]
Prediction: [[2.8810448e-06 9.9999714e-01 4.5912438e-09 2.6018718e-15]]
Prediction: [[3.64504963e-01 6.35426998e-01 6.80310768e-05 1.38153995e-08]]
Prediction: [[0.48221657 0.50044405 0.01561328 0.00172616]]
Prediction: [[5.1425326e-01 4.8542616e-01 3.2026210e-04 3.8796955e-07]]
Prediction: [[0.31285465 0.36477908 0.30982915 0.01253714]]
Prediction: [[7.9914555e-02 9.2008549e-01 4.4223643e-08 4.1470064e-13]]
Prediction: [[5.2594650e-01 4.7283831e-01 1.2150103e-03 2.3653530e-07]]
Prediction: [[1.2429169e-03 9.9875712e-01 9.0664368e-13 1.9886237e-15]]
Prediction: [[0.42239967 0.31767648 0.25895786 0.00096599]]
Prediction: [[5.8484684e-08 1.000000

Prediction: [[0.28404793 0.33238885 0.25047266 0.13309053]]
Prediction: [[0.25363344 0.25438583 0.24277161 0.2492091 ]]
Prediction: [[4.2234585e-01 5.4505533e-01 3.2518886e-02 8.0002414e-05]]
Prediction: [[0.33717844 0.3148548  0.23400041 0.11396631]]
Prediction: [[4.3734640e-01 5.6244850e-01 2.0503523e-04 8.0473058e-08]]
Prediction: [[4.8233956e-01 5.1693386e-01 7.2440185e-04 2.1839219e-06]]
Prediction: [[0.3208238  0.3053944  0.2594137  0.11436805]]
Prediction: [[4.1417739e-01 5.8582228e-01 4.5361267e-07 7.0215098e-11]]
Prediction: [[3.8157162e-01 4.4688457e-01 1.7148790e-01 5.5906934e-05]]
Prediction: [[0.35706183 0.4336425  0.20585784 0.00343772]]
Prediction: [[6.93845481e-10 1.00000000e+00 1.27772786e-14 9.60018951e-24]]
Prediction: [[5.3132492e-01 4.6641526e-01 2.2541459e-03 5.6607018e-06]]
Prediction: [[3.4236646e-01 6.5617204e-01 1.4594266e-03 1.9357940e-06]]
Prediction: [[0.26757383 0.2565835  0.21810597 0.25773665]]
Prediction: [[3.9477146e-01 6.0311705e-01 2.1110631e-03 4.18

Prediction: [[5.1767379e-01 4.8210761e-01 2.1790878e-04 7.2886058e-07]]
Prediction: [[2.6974106e-01 7.3017687e-01 8.2009901e-05 4.0471049e-09]]
Prediction: [[1.3465913e-02 9.8653412e-01 2.5506781e-08 6.9785302e-12]]
Prediction: [[0.43436778 0.33753347 0.2264914  0.00160728]]
Prediction: [[0.2444836  0.27895316 0.29094815 0.18561506]]
Prediction: [[4.1078609e-01 4.0482903e-01 1.8433215e-01 5.2649979e-05]]
Prediction: [[3.4304897e-03 9.9656928e-01 1.9983284e-07 1.1662460e-11]]
Prediction: [[8.076365e-06 9.999919e-01 3.971447e-08 4.278073e-14]]
Prediction: [[4.0889106e-04 9.9959117e-01 1.0104121e-11 2.6470582e-15]]
Prediction: [[4.0555635e-01 3.6638299e-01 2.2793987e-01 1.2074915e-04]]
Prediction: [[4.7364505e-05 9.9995267e-01 4.9173377e-13 8.6263721e-18]]
Prediction: [[0.25553143 0.275869   0.27495602 0.19364356]]
Prediction: [[4.3171546e-01 5.6825072e-01 3.3781205e-05 6.5514705e-08]]
Prediction: [[0.32716238 0.308459   0.2519366  0.11244193]]
Prediction: [[5.0012165e-01 4.9965516e-01 2.

Prediction: [[1.2280944e-02 9.8771590e-01 3.2008277e-06 3.0200700e-10]]
Prediction: [[1.2353873e-03 9.9876463e-01 4.1222394e-12 3.3491183e-17]]
Prediction: [[0.3841886  0.38673925 0.21379937 0.0152728 ]]
Prediction: [[4.6136683e-01 4.1796240e-01 1.2039682e-01 2.7390308e-04]]
Prediction: [[1.6522335e-05 9.9998295e-01 4.7220797e-07 1.6098473e-11]]
Prediction: [[0.26178113 0.2662665  0.2482799  0.2236725 ]]
Prediction: [[3.5159034e-01 6.4826208e-01 1.4752751e-04 4.8041775e-09]]
Prediction: [[3.4341606e-01 6.5601420e-01 5.6930876e-04 4.1142573e-07]]
Prediction: [[2.1235331e-07 9.9999976e-01 1.0557597e-10 3.9662653e-18]]
Prediction: [[5.5275613e-01 4.4724363e-01 2.0965892e-07 9.3779290e-12]]
Prediction: [[0.27074176 0.25783107 0.21720111 0.25422603]]
Prediction: [[4.7914079e-01 5.2032465e-01 5.3046562e-04 4.1081589e-06]]
Prediction: [[4.4526288e-01 5.2020115e-01 3.4452643e-02 8.3347622e-05]]
Prediction: [[3.9345966e-04 9.9959463e-01 1.1903279e-05 3.0069132e-09]]
Prediction: [[3.6207712e-01 

Prediction: [[3.0762008e-01 6.9233173e-01 4.8171110e-05 2.5612739e-08]]
Prediction: [[2.8996624e-03 9.9710000e-01 3.1369723e-07 6.8521907e-12]]
Prediction: [[1.4313548e-04 9.9985683e-01 5.7823673e-13 6.9696088e-18]]
Prediction: [[0.3478825  0.3418987  0.30856004 0.00165871]]
Prediction: [[1.5822990e-01 8.4176922e-01 8.7191796e-07 5.1023941e-09]]
Prediction: [[5.8555978e-09 1.0000000e+00 8.8462506e-13 1.1008446e-21]]
Prediction: [[4.9012214e-01 5.0978053e-01 9.7228891e-05 1.4109874e-07]]
Prediction: [[3.4041038e-01 6.5936095e-01 2.2857802e-04 7.4392460e-08]]
Prediction: [[0.46466756 0.37311387 0.1557839  0.00643465]]
Prediction: [[8.96990895e-02 9.10197616e-01 1.03251674e-04 9.96370719e-09]]
Prediction: [[2.6652706e-01 7.3320276e-01 2.7008980e-04 1.1050496e-07]]
Prediction: [[3.8899970e-01 3.5860974e-01 2.5228712e-01 1.0344083e-04]]
Prediction: [[0.41100487 0.3829235  0.20388523 0.00218636]]
Prediction: [[5.3438777e-01 4.6377400e-01 1.8367363e-03 1.5095776e-06]]
Prediction: [[0.4072251 

...

Prediction: [[6.904085e-08 9.999999e-01 4.198433e-15 4.910597e-24]]
Prediction: [[1.1572961e-06 9.9999881e-01 3.8648040e-10 1.7161871e-18]]
Prediction: [[0.25268784 0.3604104  0.384608   0.00229376]]
Prediction: [[2.6246268e-04 9.9972695e-01 1.0631877e-05 4.6279428e-10]]
Prediction: [[5.6905031e-01 4.3026286e-01 6.8209518e-04 4.7629137e-06]]
Prediction: [[4.9558780e-01 5.0380653e-01 5.8801210e-04 1.7709879e-05]]
Prediction: [[2.1724401e-04 9.9978250e-01 1.7924476e-07 2.2439934e-13]]
Prediction: [[2.2632976e-01 7.7367026e-01 1.3812272e-08 1.7618668e-10]]
Prediction: [[0.34580737 0.36079335 0.2910951  0.00230416]]
Prediction: [[8.0359131e-02 9.1964012e-01 6.6206530e-07 3.7308837e-10]]
Prediction: [[0.26885074 0.25542018 0.22612226 0.24960679]]
Prediction: [[1.9471402e-03 9.9783462e-01 2.1823183e-04 4.5485556e-11]]
Prediction: [[4.5841751e-01 5.3847921e-01 3.1023691e-03 9.5293859e-07]]
Prediction: [[0.2670944  0.25799808 0.22307235 0.25183523]]
Prediction: [[4.6995482e-01 5.2988756e-01 1.

Prediction: [[9.6037677e-03 9.9039626e-01 4.6327115e-09 7.5875151e-14]]
Prediction: [[7.2684238e-04 9.9915683e-01 1.1633876e-04 6.3639448e-11]]
Prediction: [[0.46866512 0.38316765 0.14698958 0.00117763]]
Prediction: [[5.0439841e-01 4.9536604e-01 2.3453134e-04 9.6491431e-07]]
Prediction: [[4.8392850e-01 5.1596892e-01 1.0251621e-04 3.8399477e-08]]
Prediction: [[4.3034256e-01 3.4033370e-01 2.2915062e-01 1.7306702e-04]]
Prediction: [[4.8529735e-01 5.1410580e-01 5.6942669e-04 2.7372831e-05]]
Prediction: [[2.3822764e-05 9.9997509e-01 1.1183957e-06 1.4290998e-10]]
Prediction: [[3.4502906e-01 6.5457183e-01 3.9883846e-04 3.1230860e-07]]
Prediction: [[1.7682508e-03 9.9823183e-01 8.9369010e-09 2.1461706e-12]]
Prediction: [[0.38934457 0.4912678  0.11002178 0.00936573]]
Prediction: [[4.4371417e-01 4.9552196e-01 6.0592841e-02 1.7107265e-04]]
Prediction: [[0.30028906 0.2712428  0.31640178 0.11206637]]
Prediction: [[0.35510856 0.35927683 0.24639167 0.03922295]]
Prediction: [[9.6613151e-04 9.9900633e-0

Prediction: [[5.3291935e-01 4.6578810e-01 1.2915739e-03 1.0785560e-06]]
Prediction: [[5.0781518e-01 4.9186549e-01 3.1927638e-04 7.5147213e-08]]
Prediction: [[0.37937433 0.3643729  0.25518557 0.00106723]]
Prediction: [[5.3916299e-01 4.5959541e-01 1.2360803e-03 5.5302075e-06]]
Prediction: [[3.5445038e-03 9.9645555e-01 1.9407544e-12 2.0007007e-15]]
Prediction: [[0.266383   0.29535663 0.24605325 0.19220714]]
Prediction: [[2.6037859e-11 1.0000000e+00 1.5916954e-19 3.8696819e-30]]
Prediction: [[3.9683622e-03 9.9603158e-01 8.1969125e-09 2.3453166e-13]]
Prediction: [[0.34473547 0.3048556  0.34683046 0.00357842]]
Prediction: [[0.304295   0.3085862  0.3292268  0.05789195]]
Prediction: [[4.7788242e-01 5.2039176e-01 1.7240790e-03 1.7420111e-06]]
Prediction: [[3.6898148e-01 4.1584828e-01 2.1493153e-01 2.3875112e-04]]
Prediction: [[2.1575704e-04 9.9978262e-01 1.6246839e-06 5.6623048e-12]]
Prediction: [[0.35284284 0.3095785  0.3358673  0.00171134]]
Prediction: [[5.0795078e-01 4.8594141e-01 6.1047636e

Prediction: [[3.0377606e-01 6.9622386e-01 7.9589419e-08 1.4182240e-10]]
Prediction: [[2.7500659e-01 7.2499341e-01 2.6999725e-08 1.8799638e-10]]
Prediction: [[0.26858467 0.2700767  0.29475403 0.16658457]]
Prediction: [[0.27333295 0.23975736 0.2646707  0.22223905]]
Prediction: [[4.0581682e-01 5.8114851e-01 1.2978517e-02 5.6113178e-05]]
Prediction: [[5.0090742e-01 4.9858582e-01 4.8823853e-04 1.8657873e-05]]
Prediction: [[5.0427461e-01 4.9545383e-01 2.7109624e-04 3.9491863e-07]]
Prediction: [[3.4914279e-07 9.9999964e-01 1.3039932e-09 4.5883155e-16]]
Prediction: [[4.4063571e-01 5.5542266e-01 3.9409464e-03 6.9482866e-07]]
Prediction: [[4.2970996e-09 1.0000000e+00 3.5949776e-14 4.7393718e-22]]
Prediction: [[9.3924656e-04 9.9906081e-01 5.2621410e-12 1.1575694e-16]]
Prediction: [[4.9888006e-01 4.9963257e-01 1.4866489e-03 7.6925522e-07]]
Prediction: [[0.43544096 0.3639483  0.198321   0.00228984]]
Prediction: [[3.4990352e-01 3.3702201e-01 3.1285760e-01 2.1689772e-04]]
Prediction: [[4.8113415e-01 

Prediction: [[0.38698202 0.31999183 0.29180247 0.00122365]]
Prediction: [[2.0995454e-01 7.8916794e-01 8.7625301e-04 1.2767186e-06]]
Prediction: [[1.7522252e-04 9.9982482e-01 1.4975037e-11 4.9038983e-16]]
Prediction: [[0.32037097 0.30297855 0.25452718 0.12212332]]
Prediction: [[4.8631108e-01 5.0894964e-01 4.7371727e-03 2.0581049e-06]]
Prediction: [[4.9842173e-01 4.9998504e-01 1.5906888e-03 2.5020834e-06]]
Prediction: [[4.70689148e-01 4.21923846e-01 1.07367925e-01 1.90734645e-05]]
Prediction: [[5.3369085e-04 9.9946636e-01 4.5995150e-09 6.6941497e-15]]
Prediction: [[4.7562850e-01 5.2088183e-01 3.4660010e-03 2.3651563e-05]]
Prediction: [[0.47394428 0.43430334 0.09091027 0.00084217]]
Prediction: [[5.3428411e-01 4.6343893e-01 2.1934628e-03 8.3554842e-05]]
Prediction: [[5.0341332e-01 4.9624357e-01 3.4260677e-04 5.1585039e-07]]
Prediction: [[5.8671623e-04 9.9941289e-01 4.0820703e-07 4.5577490e-11]]
Prediction: [[4.9321586e-01 5.0365871e-01 3.1239716e-03 1.4329562e-06]]
Prediction: [[0.29915038

In [19]:
values = X.iloc[10].values
values.shape


values = values.reshape(1,-1)
values_scaled = X_scaler.transform(values)
values_scaled


values_scaled.shape


this = model.predict(values_scaled)
that = f"{this[0][1]*100}%"
print(that)

41.44834280014038%


In [20]:
model.save("../Model_Info/neuralnetwork.h5")



In [21]:
cancer_model = load_model("../Model_Info/neuralnetwork.h5")

In [22]:
model_loss, model_accuracy = cancer_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 0.7776988447586463, Accuracy: 0.48328445747800586


In [23]:
values = X.iloc[10].values
values.shape


values = values.reshape(1,-1)
values_scaled = X_scaler.transform(values)
values_scaled


values_scaled.shape


this = model.predict(values_scaled)
that = f"{this[0][1]*100}%"
print(that)

41.44834280014038%


In [24]:
values_scaled

array([[ 6.40543676, -0.25687714, -1.84059791,  1.37272516,  0.98869536,
         2.70845033,  0.92754841, -0.23052352, -0.32812951, -0.58691132,
        -0.16279036,  0.86800308, -0.89497836,  0.89497836, -0.10037464,
         7.11663296, -0.12283463, -0.16530486, -0.21850836, -0.13759304,
        -0.1300205 , -0.11169265, -0.19752585, -0.15895185, -0.11169265,
        -0.1169599 , -0.08653141, -0.17681235, -0.15033296, -0.11609777,
        -0.14267172, -0.14051589, -0.15502789, -0.16716852, -0.0983674 ,
        -0.1888099 , -0.15502789, -0.14619969, -0.11522953, -0.10331528,
        -0.1888099 , -0.08059816, -0.10988369, -0.09209377, -0.17622329,
        -0.08879685, -0.13156743, -0.19537807, -0.17973217, -0.14619969,
        -0.14409243, -0.18545099, -0.09422973, -0.14195646, -0.09316766,
        -0.14965133, -0.19047016, -0.09316766, -0.16592816,  0.        ,
        -0.09209377, -0.18375162, -0.15236118, -0.12446509, -0.07552596]])

In [25]:
values



array([[12.05      ,  0.        ,  0.        ,  0.08822222, 29.07692308,
        19.16      ,  5.2       ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])

In [30]:
values

array([[12.05      ,  0.        ,  0.        ,  0.08822222, 29.07692308,
        19.16      ,  5.2       ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])